# Database Exploration

By **Franklin Oliveira**

-----

This notebook contains some code written to I (Franklin) get accquainted with the `repteis` database. Here you'll find some basic data treatment and adjustments that presented necessary as I started to understand the nature of the information in file <font color='blue'>'Compilacao Livros Repteis - 2 a 10 - 2020_04_28.xls'</font>.

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# pacotes para visualização rápida
import seaborn as sns
import matplotlib.pyplot as plt

# pacote para visualização principal
import altair as alt

# habilitando renderizador para notebook
# alt.renderers.enable('notebook')
alt.renderers.enable('default')


# desabilitando limite de linhas
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [2]:
excel = pd.ExcelFile('Compilacao Livros Repteis - 2 a 10 - 2020_04_28.xls')
sheet_name = excel.sheet_names

print('The excel file contains the following sheets:', sheet_name)
print('\nDatabase is in sheet:', sheet_name[0])

The excel file contains the following sheets: ['Repteis-2020-02-11-csv']

Database is in sheet: Repteis-2020-02-11-csv


In [3]:
db = excel.parse(sheet_name[0], sep=';', encoding='utf-8-sig')

# p.s.: I'm parsing a pre-treated file provided by Asla
#db = pd.read_excel('db.csv', sep=',', encoding='utf-8-sig', low_memory=False)
print(f'The database has {db.shape[0]} rows and {db.shape[1]} columns.')

The database has 23119 rows and 114 columns.


In [4]:
# copying database to another variable to make a few changes keeping the original intact
repteis = db.copy()

<br>

<font color='red' size='5'>**p.s.:** We were told by the Curator that some names in this database may remain confidential. So, I'm gonna skip this step for now until we know how to treat this data appropriately </font>

### Name columns

`DeterminatorFirstName1` e `DeterminatorLastName1`

#### treating determiner's name 

In this step, we're going to put together 'determinator's' first and last name. 

`DeterminatorFirstName1` + `DeterminatorLastName1`

Just concatenating those two columns is not satisfactory because there's some dirt in these data columns. So, let's begin by treating them...

In [5]:
# repteis['DeterminatorFirstName1'].value_counts()
# repteis['DeterminatorLastName1'].value_counts()

In [6]:
def treat_names(name, pos='first'):
    '''
    Treat names keeping NaN as such.
    
    Arguments: 
        - name: name to be treated. 
        - pos (str): name position. One of ['first', 'last']
    '''
    if type(name) == str and pos == 'first':     # first name
        
        if len(name.split(' ')) > 1:             # treats composite names (+ 1 name)
            return str(name).strip().split(' ')[0].capitalize()
        else:
            return str(name).strip().capitalize()
    
    elif type(name) == str and pos == 'last':    # last name
        
        if len(name.split(' ')) > 1:             # treats composite last name (+ 1 surname)
            return str(name).strip().split(' ')[-1].capitalize()
        else:
            return str(name).strip().capitalize()   
    else:
        return name

applying function to the two column names

In [7]:
#crustaceas['Determiner First Name1'] = crustaceas['Determiner First Name1'].apply(treat_names)
# crustaceas['Determiner First Name1'].value_counts()

In [8]:
#crustaceas['Determiner Last Name1'] = crustaceas['Determiner Last Name1'].apply(lambda x:
#                                                                treat_names(x, pos='last'))
# crustaceas['Determiner Last Name1'].value_counts()

In [9]:
# creating column with First and Last name for identification
#crustaceas['Determiner First_and_Last Name'] = crustaceas['Determiner First Name1'] + ' ' + crustaceas['Determiner Last Name1']

<br>

## Adjusting columns names

### removing '\n'

In [10]:
repteis.columns = [str(col).replace(r'\n','') for col in repteis.columns]

#### closer look on some columns...

In [11]:
for col in repteis.columns:
    print('-', col)

- NumeroDeCatalogo
- NumeroDeCampo
- DataDeEntrada
- DataDaDeterminacao
- Kingdom
- Phylum
- Class
- Ordem
- Familia
- Genero_ent
- Qualificador_ent
- Especie_ent
- Subespecie_ent
- Type Status 1
- Current 1
- Determined Date 2
- Class2
- Order2
- Family2
- Genero_atual
- Qualificador_atual
- Especie_atual
- Subespecie_atual
- NotasTaxonomicas
- Current 2
- DeterminatorLastName1
- DeterminatorFirstName1
- DeterminatorMiddleInitial1
- DeterminatorLastName2
- DeterminatorFirstName2
- DeterminatorMiddleInitial2
- DeterminationRemark
- AssociatedTaxa
- TypeOf
- ColecaoEspecial
- DataColetaInicial
- DataColetaFinal
- Complemento
- NomeDaLocalidade
- Municipio
- EstadoOuProvincia
- Pais
- Continente
- LocalityRemark
- UtmDatum
- UtmZone
- UtmEasting
- UtmNorthing
- VerbatimLatitude
- VerbatimLongitude
- Lat
- Long
- PrecisaoDaCoordenada
- GrupoDeColeta
- MinAltitude
- MaxAltitude
- CollectingInformationRemark
- CollectorLastName1
- CollectorFirstName1
- CollectorMiddleName1
- CollectorLastNa

In [12]:
repteis['Especie_atual'].value_counts()

torquatus     1403
ocellifera     779
jararaca       635
hispidus       607
ameiva         558
              ... 
notaeus          1
eladioi          1
dorbigny         1
werneri          1
insularis        1
Name: Especie_atual, Length: 803, dtype: int64

<font size='5'>**Equivalência de colunas:** </font>

**Nome diferente:** <br>
- Species1: Especie_ent ou Especie_atual
- Species Author1: ?
- Type Status1: Type Status 1
- Qualifier1: Qualificador_ent Qualificador_atual
- Determiner First Name1: DeterminatorFirstName1
- Determiner Middle1: DeterminatorMiddleInitial1
- Determiner Last Name1: DeterminatorLastName1
- Determined Date1: DataDaDeterminacao

In [13]:
repteis[['Especie_atual', 'Type Status 1', 'Qualificador_atual', 'DeterminatorFirstName1',
       'DeterminatorMiddleInitial1', 'DeterminatorLastName1', 'DataDaDeterminacao']]

,Especie_atual,Type Status 1,Qualificador_atual,DeterminatorFirstName1,DeterminatorMiddleInitial1,DeterminatorLastName1,DataDaDeterminacao
0,geoffroanus,NaN,-,L.,M.F.,Cunha,00/11/2018
1,suspectum,NaN,NaN,NaN,NaN,NaN,NaN
2,alba,NaN,NaN,NaN,NaN,NaN,NaN
3,corais,NaN,NaN,NaN,NaN,NaN,NaN
4,bottae,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
23114,ibiboboca,NaN,NaN,P.,NaN,Passos,00/10/2016
23115,ibiboboca,NaN,NaN,P.,NaN,Passos,00/10/2016
23116,quadricarinatus,NaN,NaN,P.,NaN,Passos,00/10/2016
23117,bicarinatus,NaN,NaN,P.,NaN,Pinna,00/10/2016


<br>

## preparing data for charts...

### Column: `Type Status1`

contains info on species type

In [14]:
repteis['Type Status 1'].value_counts().head()  # essa coluna está vazia!?

Series([], Name: Type Status 1, dtype: int64)

#### Let's begin cleaning this data and lowering all cases.

In [15]:
#repteis['Type Status 1'] = repteis['Type Status 1'].str.strip().str.lower().str.capitalize()

In [16]:
repteis['Type Status 1'].value_counts().head()

Series([], Name: Type Status 1, dtype: int64)

### preparing taxonomy columns

`Kingdom` - `Phylum` - `Class` - `Ordem` - `Familia` - `Genero_ent`- `Genero_atual` - `Especie_ent` - `Especie_atual`

**Colunas faltando:**
- `Subphylum1`
- `Subclass1`
- `Infraclass1`
- `Superorder1`
- `Suborder1` 
- `Infraorder1` 
- `Superfamily1`
- `Subfamily1` 
- `Tribe1`

In [17]:
taxon_columns = ['Kingdom', 'Phylum', 'Class', 'Ordem', 'Familia', 'Genero_ent',
                 'Genero_atual', 'Especie_ent', 'Especie_atual', 'Subespecie_ent',
                 'Subespecie_atual']  # selecting taxonomy columns

# defining function
def treat_str(x):
    return str(x).lower().capitalize().strip()

# applying treatment
for col in taxon_columns:
    print(f'Adjusting column {col}')
    repteis[col] = repteis[col].apply(treat_str)

Adjusting column Kingdom
Adjusting column Phylum
Adjusting column Class
Adjusting column Ordem
Adjusting column Familia
Adjusting column Genero_ent
Adjusting column Genero_atual
Adjusting column Especie_ent
Adjusting column Especie_atual
Adjusting column Subespecie_ent
Adjusting column Subespecie_atual


### adding `Genero` and `Especie`together (they completely identify each animal's species)

In [18]:
repteis['genero_e_especie_ent'] = repteis['Genero_ent'] + ' ' + repteis['Especie_ent']
repteis['genero_e_especie_atual'] = repteis['Genero_atual'] + ' ' + repteis['Especie_atual']

repteis['genero_e_especie_ent'] = repteis['genero_e_especie_ent'].str.lower().str.capitalize()
repteis['genero_e_especie_atual'] = repteis['genero_e_especie_atual'].str.lower().str.capitalize()

<br>

### Collecting date (year) and sclicing main DB to a smaller dataset 

Columns: `Determined Date1` - `Class1` - `Kingdom` and more...

In [19]:
# slicing main database (repteis)
Table = repteis[['DataDeEntrada','DataDaDeterminacao','DataColetaInicial','Class','Kingdom', 
                    'Genero_ent', 'Genero_atual', 'Especie_ent', 'Especie_atual', 'Type Status 1',
                    'DeterminatorFirstName1', 'DeterminatorLastName1', 'genero_e_especie_ent',
                    'genero_e_especie_atual','MinAltitude',
                    'Ordem', 'Familia', 'Phylum']].copy()

# OBS: Determined Date1 has many missing values... CHECK THAT
d = []
counter=0
for row in Table['DataColetaInicial']:
    if not str(row).find('/')==-1:
        dates_values = str(row).split("/")
        year = int(dates_values[-1])
        month = int(dates_values[1])
#        if (month>1) and (month<12):
            #store the year and month in a datetime datatype for later sorting
#            dateRecord = datetime.datetime(year,month,1) 
    else:
        year = Table.loc[counter, 'DataColetaInicial']
    
    # mais um condicional para tratar anos vazios ' '
    if year == ' ':
        year = np.NAN
        
    d.append({'ano_coleta':year,
              'class':Table.loc[counter,'Class'],
              'kingdom':Table.loc[counter,'Kingdom'], 'genero_ent':Table.loc[counter,'Genero_ent'],
              'genero_atual':Table.loc[counter,'Genero_atual'],
              'especie_ent':Table.loc[counter,'Especie_ent'],
              'especie_atual':Table.loc[counter,'Especie_atual'],
              'genero_e_especie_ent': Table.loc[counter,'genero_e_especie_ent'],
              'genero_e_especie_atual': Table.loc[counter,'genero_e_especie_atual'],
              'type_status':Table.loc[counter,'Type Status 1'], 
              'determinator_first_name':Table.loc[counter,'DeterminatorFirstName1'],
              'determinator_last_name':Table.loc[counter,'DeterminatorLastName1'],
              'altitude':Table.loc[counter,'MinAltitude'],
              'ordem':Table.loc[counter,'Ordem'],
              'familia':Table.loc[counter,'Familia'],
              'phylum': Table.loc[counter,'Phylum']
              })
    counter = counter+1

    
NewTable = pd.DataFrame(d)


### collecting determined year (p.s.: being careful to keep NaNs as they show up)
NewTable['ano_determinacao'] = np.nan

d1 = []
counter=0
for row in Table['DataDaDeterminacao']:
    try:  # if Determined Date1 is empty, keep it so 
        if np.isnan(row):
            year= np.NAN
    
    except:
        if not str(row).find('/')==-1:
            dates_values = str(row).split("/")
            year = int(dates_values[-1])
            month = int(dates_values[1])
#            if (month>1) and (month<12):
                #store the year and month in a datetime datatype for later sorting
#                dateRecord = datetime.datetime(year,month,1)    
    
    NewTable.loc[counter, 'ano_determinacao'] = year
    counter = counter+1

    
### collecting start year (p.s.: being careful to keep NaNs as they show up)  
NewTable['ano_entrada'] = np.nan
d1 = []
counter=0
for row in Table['DataDeEntrada']:
    try:  # if Start Date is empty, keep it so 
        if np.isnan(row):
            year= np.NAN
    
    except:
        if not str(row).find('/')==-1:
            dates_values = str(row).split("/")
            year = int(dates_values[-1])
            month = int(dates_values[1])
#            if (month>1) and (month<12):
                #store the year and month in a datetime datatype for later sorting
#                dateRecord = datetime.datetime(year,month,1)    

    NewTable.loc[counter, 'ano_entrada'] = year
    counter = counter+1

# NewTable['determined_year'] = pd.Series(year, index=NewTable.index)
NewTable.head(2)

,ano_coleta,class,kingdom,genero_ent,genero_atual,especie_ent,especie_atual,genero_e_especie_ent,genero_e_especie_atual,type_status,determinator_first_name,determinator_last_name,altitude,ordem,familia,phylum,ano_determinacao,ano_entrada
0,NaN,Reptilia,Animalia,Nan,Phrynops,Nan,Geoffroanus,Nan nan,Phrynops geoffroanus,NaN,L.,Cunha,NaN,Testudines,Chelidae,Chordata,2018.0,NaN
1,NaN,Reptilia,Animalia,Heloderma,Heloderma,Suspectum,Suspectum,Heloderma suspectum,Heloderma suspectum,NaN,NaN,NaN,NaN,Squamata,Helodermatidae,Chordata,NaN,NaN


In [20]:
# checks if NaNs are in the same position 
result = (NewTable['ano_entrada'].isna() == repteis['DataDeEntrada'].isna()).sum() == NewTable.shape[0]

if result:
    print('ano_entrada info is valid.')
else:
    print("There's something wrong with NewTable. Check how you're collecting Start Year info.")

ano_entrada info is valid.


In [21]:
# year in which the holotipo was "firstly" found 
#NewTable['holotipo_year'] = NewTable['species_author'].str.extract('(\d+)')

<br>

### creating `years` columns in repteis

In [22]:
def catch_year(row):
    if not str(row).find('/')==-1:
        dates_values = str(row).split("/")
        year = int(dates_values[-1])
        month = int(dates_values[1])
        return year
    else:
        return np.NaN

In [23]:
repteis['ano_determinacao'] = repteis['DataDaDeterminacao'].apply(catch_year)
repteis['ano_coleta'] = repteis['DataColetaInicial'].apply(catch_year)
repteis['ano_entrada'] = repteis['DataDeEntrada'].apply(catch_year)

<br>

---

## Graphs

<font color='red'>**p.s.:** All the main charts were moved to individual notebooks. I've kept these two below only in case we need them.</font>

### Total amount of catalogations per year

x: Start Year (from Start Date)
y: number of catalogations per year

In [24]:
# counting catalog. per year
teste = repteis['ano_coleta'].value_counts()
teste = teste.reset_index().rename(columns={'index':'year', 'ano_coleta':'counts'})

In [25]:
# adjusting columns for graphs
teste['year'] = teste['year'].apply(lambda x:str(x).split('/')[0].split('.')[0]).astype(int)
teste = teste.groupby('year').sum().reset_index() # soma do total de bichos coletados por ano

In [26]:
# min e max para eixo X (year)
min_x = teste['year'].min()
max_x = teste['year'].max()

In [27]:
temp = alt.Chart(data= teste, width=800, title= 'Number of collected animals per year').mark_bar().encode(
    x= alt.X('year', type='ordinal', title='Ano de Coleta'),
    y= alt.Y('counts', type='quantitative', title='Contagem')
)

temp.save('./graphs/coletas_por_ano.html')
temp

alt.Chart(...)

<br>

---

### Types (*per year*) <font color='red'>Coluna Tipos n tá preenchida. Trocando por Espécie...</font>

#### adjusting columns `determined_year` e `cataloged_year` to ```*int*``` format 

In [28]:
def str_with_nan2int(string):
    if not np.isnan(string):
        return int(string)
    else:
        return np.NAN

In [29]:
NewTable['ano_determinacao'] = NewTable['ano_determinacao'].apply(str_with_nan2int) #has NaN
NewTable['ano_coleta'] = NewTable['ano_coleta'].apply(str_with_nan2int) #has NaN
NewTable['ano_entrada'] = NewTable['ano_entrada'].apply(str_with_nan2int) #has NaN

#### removing duplicated rows as we took a subset of the main database

In [30]:
teste1 = NewTable.drop_duplicates().copy()  # removes duplicated rows (with same values in ALL columns)
print('Duplicated registers:',NewTable.shape[0] - NewTable.drop_duplicates().shape[0])
# teste1.head(2)

Duplicated registers: 13835


### Number of catalogations of each type (per year)

In [31]:
# p.s.: I'm grouping again because, previously, there were other charts in this notebook. 
# Now, I'm only keeping what's important for this chart (in case we need to get back to it)

# p.s.: try changing year column in groupby to start_year
type_data = teste1.groupby(['altitude','familia']).count()['class'].reset_index().rename(
                                                    columns={'class':'counts'})

type_data.sort_values(['counts'], inplace=True) # sorting...

In [32]:
teste = type_data
#teste = NewTable.groupby('altitude').count()['class'].reset_index().rename(columns={'class':'counts'})

In [33]:
teste = NewTable[['altitude','familia','ordem']].copy()

In [34]:
teste['altitude'] = teste['altitude'].str.extract('(\d+)')

In [35]:
teste = teste.sort_values(['altitude','familia'])

In [36]:
temp = alt.Chart(teste, title='Altitude por Familia').mark_circle().encode(
    x = alt.X('familia', type='nominal', title='Familia'),
    y = alt.Y('altitude', type='quantitative', title='Altitude (em metros)')
)

temp.save('./graphs/altitude_por_familia.html')
temp

alt.Chart(...)

In [37]:
# p.s.: I'm grouping again because, previously, there were other charts in this notebook. 
# Now, I'm only keeping what's important for this chart (in case we need to get back to it)

# p.s.: try changing year column in groupby to start_year
type_data = teste1.groupby(['ano_coleta','ordem']).count()['class'].reset_index().rename(
                                                    columns={'class':'counts'})

type_data.sort_values(['counts'], inplace=True) # sorting...

In [38]:
# p.s.: try changing X axis to start_year ;)
g3 = alt.Chart(type_data, title='Number of animals of each order per year').mark_rect().encode(
    x= alt.X('ano_coleta', type='ordinal', title='Year'),
    y= alt.Y('ordem', type='nominal', title='Ordem', 
             sort=alt.EncodingSortField(field="counts", op="max", order='descending'),
            axis=alt.Axis(grid=False)),
    color= alt.Color('counts', type='quantitative', scale=alt.Scale(scheme='yellowgreenblue'))
)

g3.save('./graphs/ordens_por_ano.html')
# g3.save('./viz_old/types_per_year.svg')

g3

alt.Chart(...)

<br>

**Thats it!**

-----